In [1]:
# %pip install pandas
# %pip install catboost
# %pip install scikit-learn

In [2]:
import pandas as pd
from catboost import CatBoostRanker, Pool
from sklearn.model_selection import StratifiedGroupKFold
from time import time

RANDOM_STATE = 42

# Анализ данных

In [3]:
# Загрузим данные
df_train = pd.read_csv('data/train_df.csv')
df_test = pd.read_csv('data/test_df.csv')

In [4]:
# Удалим дубликаты
df_train.drop_duplicates(inplace=True)

In [5]:
#Посмотрим на датасет
df_train.sample(10)

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
10645,76443,9,0,0,0,9,4,47,0,2,...,0.315400,0.301275,0.095022,0,0,0,0.60546,0.0,0.0,0
11991,202264,9,1,2,1,20,4,31,1,1,...,0.221742,0.716336,0.158842,0,0,0,0.17985,0.0,0.0,0
8811,396025,9,0,0,0,9,4,28,0,0,...,0.300139,0.810031,0.243122,0,0,0,0.49742,0.0,0.0,0
889,105302,9,0,0,1,20,4,26,0,0,...,0.235010,0.361319,0.084914,0,0,0,0.10866,0.0,0.0,0
14001,406434,9,0,0,1,20,3,49,0,6,...,0.229567,0.477006,0.109505,0,0,0,0.17614,0.0,0.0,0
10176,25415,9,0,0,1,20,3,46,0,0,...,0.200205,0.199302,0.039901,0,0,0,0.76391,0.0,0.0,0
6201,148390,9,0,2,0,9,4,34,0,0,...,0.244329,0.347375,0.084874,0,0,0,0.07308,0.0,0.0,0
4033,413779,9,0,0,0,9,4,44,0,0,...,0.176919,0.513340,0.090820,0,0,0,0.17614,0.0,0.0,0
3464,346401,9,0,1,1,20,4,25,7,3,...,0.184892,0.618260,0.114311,0,0,0,0.10866,0.0,0.0,0
10749,82198,9,0,5,1,20,4,40,0,2,...,0.299663,0.460338,0.137946,0,0,0,0.47926,0.0,0.0,0


Оценки ранжирования представлены бинарными значениями 0 и 1.

In [6]:
df_train.describe()

,search_id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,feature_70,feature_71,feature_72,feature_73,feature_74,feature_75,feature_76,feature_77,feature_78,target
count,15069.000000,15069.0,15069.00000,15069.000000,15069.000000,15069.000000,15069.000000,15069.000000,15069.000000,15069.000000,...,15069.000000,15069.000000,15069.000000,15069.0,15069.0,15069.0,15069.000000,15069.000000,15069.000000,15069.000000
mean,254801.947641,9.0,0.19351,0.463999,0.630168,15.931847,3.876302,33.807685,1.210764,1.792023,...,0.199704,0.426882,0.096538,0.0,0.0,0.0,0.297702,0.030567,0.031232,0.021368
std,142377.307514,0.0,0.62605,1.182782,0.482775,5.310526,0.469923,11.312467,2.234444,1.883771,...,0.088580,0.247193,0.063289,0.0,0.0,0.0,0.311973,0.113026,0.115641,0.144614
min,116.000000,9.0,0.00000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000
25%,134823.000000,9.0,0.00000,0.000000,0.000000,9.000000,4.000000,26.000000,0.000000,0.000000,...,0.168716,0.255757,0.052735,0.0,0.0,0.0,0.036740,0.000000,0.000000,0.000000
50%,255859.000000,9.0,0.00000,0.000000,1.000000,20.000000,4.000000,34.000000,0.000000,2.000000,...,0.212826,0.431185,0.091782,0.0,0.0,0.0,0.176140,0.000000,0.000000,0.000000
75%,376175.000000,9.0,0.00000,0.000000,1.000000,20.000000,4.000000,44.000000,1.000000,3.000000,...,0.253881,0.603934,0.136083,0.0,0.0,0.0,0.529630,0.000000,0.000000,0.000000
max,496975.000000,9.0,3.00000,6.000000,1.000000,20.000000,6.000000,49.000000,7.000000,7.000000,...,0.485510,0.999072,0.411563,0.0,0.0,0.0,1.000020,0.846272,0.908132,1.000000


In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15069 entries, 0 to 15080
Data columns (total 81 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   search_id   15069 non-null  int64  
 1   feature_0   15069 non-null  int64  
 2   feature_1   15069 non-null  int64  
 3   feature_2   15069 non-null  int64  
 4   feature_3   15069 non-null  int64  
 5   feature_4   15069 non-null  int64  
 6   feature_5   15069 non-null  int64  
 7   feature_6   15069 non-null  int64  
 8   feature_7   15069 non-null  int64  
 9   feature_8   15069 non-null  int64  
 10  feature_9   15069 non-null  int64  
 11  feature_10  15069 non-null  int64  
 12  feature_11  15069 non-null  int64  
 13  feature_12  15069 non-null  int64  
 14  feature_13  15069 non-null  int64  
 15  feature_14  15069 non-null  int64  
 16  feature_15  15069 non-null  int64  
 17  feature_16  15069 non-null  float64
 18  feature_17  15069 non-null  float64
 19  feature_18  15069 non-null  fl

В train датасете присутствуют строки с константным значением.

In [8]:
df_train.nunique()[df_train.nunique() == 1]

feature_0     1
feature_73    1
feature_74    1
feature_75    1
dtype: int64

In [9]:
# Удалим их
df_train.drop(columns=['feature_0', 'feature_73', 'feature_74', 'feature_75'],
              inplace=True)

In [10]:
# Посмотрим, сколько на каждый запрос может быть рекомендовано документов
df_train.groupby('search_id').sum()['target'].unique().tolist()

[0, 1, 2, 4, 3, 5]

В обучающей выборке наблюдается от 0 до 5 рекомендованных значений.

In [11]:
# Посмотрим на баланс классов
100 * len(df_train[df_train['target'] == 0]['target'])/len(df_train)

97.86316278452452

Наблюдается сильный дисбаланс рекоммендаций - почти 98% значений не рекомендовано к выдаче.

In [12]:
# Посмотрим на признаки, которые имеют до 10 уникальных значений
probably_categorical_list = df_train.nunique()[df_train.nunique() < 10].index.tolist()[:-1:]
df_train[probably_categorical_list].sample(10)

,feature_1,feature_2,feature_3,feature_4,feature_5,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15
10089,0,0,1,20,4,0,0,1,0,0,0,1,1,0
5658,3,0,1,20,4,7,6,1,0,0,0,0,0,0
1960,0,0,0,9,4,7,1,0,1,1,0,0,1,0
4502,0,1,1,20,4,0,3,0,0,0,0,0,1,0
9920,0,0,1,20,4,7,5,1,0,0,0,0,1,0
7006,0,0,1,20,4,0,4,1,0,0,0,0,1,0
14642,0,0,1,20,4,6,3,1,1,1,0,0,0,0
9266,0,0,1,20,4,0,0,1,0,0,0,1,1,0
2221,0,0,1,20,4,0,4,1,0,0,0,0,0,0
7289,2,2,1,20,4,4,2,1,1,1,0,1,0,0


Признаки, которые имеют до 10 уникальных значений и тип int64, скорее всего являются категориальными
Датасет выглядит чисто, очевидных ошибок не наблюдается.

In [13]:
# Аналогично обработаем test
df_test.drop(columns=['feature_0', 'feature_73', 'feature_74', 'feature_75'],
             inplace=True)

# Построение модели

Для создания модели ранжирования воспользуемся CatBoostRanker.

In [14]:
# Отделим таргеты
X = df_train.copy()
y = X.pop('target')

X_test = df_test.copy()
y_test = X_test.pop('target')

Создадим валидационный датасет для подбора гиперпараметров. Используем стратификацию и GroupKFold для того, чтобы обеспечить одинаковое распределения таргета в тестовой и валидационной выборках, а также для того, чтобы запросы (queries) не разбивались во время сплита и целиком входили либо в train, либо в val.

In [15]:
# Создадим валидационный датасет для подбора гиперпараметров
sgkf = StratifiedGroupKFold(n_splits=5)

groups = X['search_id'].tolist()
g = sgkf.split(X, y, groups)

# Берем первый фолд - так обеспечивается распределение 80/20
# val включает в себя ~3к значений, что вдвое больше теста
train_idx, val_idx = next(g)
X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

In [16]:
# Удалим search_id
queries_train = X_train['search_id'].tolist()
queries_val = X_val['search_id'].tolist()
queries_test = X_test['search_id'].tolist()

X_train.pop('search_id')
X_val.pop('search_id')
X_test.pop('search_id')
pass

In [17]:
# Создадим пулы - структуру данных, которая используется в CatBoost
train_pool = Pool(data=X_train,
                  label=y_train,
                  group_id=queries_train,
                  cat_features=probably_categorical_list)
val_pool = Pool(data=X_val,
                label=y_val,
                group_id=queries_val,
                cat_features=probably_categorical_list)
test_pool = Pool(data=X_test,
                 label=y_test,
                 group_id=queries_test,
                 cat_features=probably_categorical_list)

Посмотрим, какая из функций потерь показывает лучший результат метрики NDCG.

In [18]:
loss_functions = ['StochasticFilter', 'PairLogit', 'PairLogitPairwise', 'LambdaMart',
                  'QueryRMSE', 'QuerySoftMax', 'YetiRank', 'YetiRankPairwise',]
results = []

# Обучим несколько ранжирующих моделей
for loss_function in loss_functions:
    t0 = time()
    rnkr = CatBoostRanker(custom_metric=['NDCG'],
                          random_state=RANDOM_STATE,
                          iterations=500,
                          loss_function=loss_function,
                          verbose=False)

    rnkr.fit(train_pool, eval_set=val_pool)
    score = round(rnkr.score(val_pool), 3)
    print(f'{loss_function}, NDCG={score}, t={round(time()-t0, 2)}')
    results.append((loss_function, score))

StochasticFilter, NDCG=0.88, t=8.38
PairLogit, NDCG=0.881, t=5.29
PairLogitPairwise, NDCG=0.869, t=73.68
LambdaMart, NDCG=0.888, t=1.91
QueryRMSE, NDCG=0.875, t=1.92
QuerySoftMax, NDCG=0.88, t=13.03
YetiRank, NDCG=0.884, t=2.87
YetiRankPairwise, NDCG=0.881, t=26.41


Наилучший результат показывает модель, использующая LambdaMart.

In [19]:
# Обучим модель с лоссом LambdaMart
rnkr = CatBoostRanker(custom_metric=['NDCG'],
                      random_state=RANDOM_STATE,
                      iterations=500,
                      loss_function='LambdaMart',
                      verbose=50)

rnkr.fit(train_pool, eval_set=val_pool)

Groupwise loss function. OneHotMaxSize set to 10
0:	test: 0.8525803	best: 0.8525803 (0)	total: 4.36ms	remaining: 2.18s
50:	test: 0.8827896	best: 0.8836569 (31)	total: 188ms	remaining: 1.65s
100:	test: 0.8850166	best: 0.8881081 (89)	total: 419ms	remaining: 1.66s
150:	test: 0.8843020	best: 0.8881081 (89)	total: 610ms	remaining: 1.41s
200:	test: 0.8849481	best: 0.8881081 (89)	total: 818ms	remaining: 1.22s
250:	test: 0.8834978	best: 0.8881081 (89)	total: 1.01s	remaining: 999ms
300:	test: 0.8822042	best: 0.8881081 (89)	total: 1.19s	remaining: 787ms
350:	test: 0.8851783	best: 0.8881081 (89)	total: 1.39s	remaining: 590ms
400:	test: 0.8834402	best: 0.8881081 (89)	total: 1.61s	remaining: 397ms
450:	test: 0.8836695	best: 0.8881081 (89)	total: 1.87s	remaining: 203ms
499:	test: 0.8838802	best: 0.8881081 (89)	total: 2.1s	remaining: 0us

bestTest = 0.8881081248
bestIteration = 89

Shrink model to first 90 iterations.


# Предсказание на тестовой выборке

Расчитаем NDCG на тесте и сохраним предсказание в файл.
Согласно документации CatBoost, метод score() класса CatBoostRanker считает именно эту метрику.

In [20]:
# NDCG на тестовом датасете
round(rnkr.score(test_pool), 2)

0.91

Мы получили довольно высокий NDCG. Альтернативно можно было бы посчитать через ndcg_score() из sklearn, но в таком случае придется группировать значения в списке по каждому запросу и обрабатывать случаи, когда в запросе нет ненулевых таргетов (такие запросы сильно понижают средний NDCG по датасету). По всей видимости, CatBoost.score() не берет в расчет эти случаи.

In [21]:
# Добавим ранжирование в тестовый датасет
predictions = pd.concat([df_test['search_id'],
                        pd.DataFrame(rnkr.predict(test_pool),
                                     columns=['ranking']),
                        df_test['target']], axis=1)
predictions

,search_id,ranking,target
0,10655,-0.470215,0
1,10655,-0.456435,0
2,10655,-0.428377,0
3,10655,-0.376507,0
4,10655,-0.448321,0
...,...,...,...
1524,493078,0.061529,0
1525,493078,-0.109732,0
1526,493078,0.011676,1
1527,493078,-0.109732,1


In [22]:
# Сохраним в файл
predictions.to_csv('predictions.txt', header=True, index=False)